In [2]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

In [3]:
data_dir = '/opt/ml/movie-recommendation/data/'
train_df = pd.read_csv(os.path.join(data_dir+'train', 'train_ratings.csv')) # 전체 학습 데이터

In [4]:
# most popular 10 items
num_items = train_df['item'].nunique()
num_users = train_df['user'].nunique()
most_popular_items = train_df['item'].value_counts()[:int(num_items*0.1)].index.values

most_popular_items = set(most_popular_items)
observed_items_per_user = list(train_df.groupby('user')['item'])


In [5]:
# 각 유저마다 안본 영화만 선택

unseen_items_dfs = list()

for user, observed_items in tqdm(observed_items_per_user):
    observed_items = set(observed_items)
    unseen_item = list(most_popular_items - observed_items)

    user_id = [user]*len(unseen_item)
    unseen_items_dfs.append(pd.DataFrame(zip(user_id,unseen_item), columns=['user','item']))

test_df = pd.concat(unseen_items_dfs, axis = 0, sort=False)

100%|██████████| 31360/31360 [00:30<00:00, 1030.71it/s]


In [7]:
test_df = test_df.sort_values(by=['user'])
test_df.reset_index(drop=True, inplace=True)
test_df

,user,item
0,11,2
1,11,3362
2,11,7451
3,11,79132
4,11,1307
...,...,...
18182955,138493,2700
18182956,138493,648
18182957,138493,2692
18182958,138493,2683


In [8]:
test_df.to_csv(data_dir+'eval/Unobserved Cases.csv', index=False)